In [1]:
def normalize(X):
    for i in range(X.shape[1]):
        X[:,i] = (X[:,i] - X[:,i].mean())/X[:,i].std()
    return X

In [2]:

# from sklearn import tree
import pandas as pd
import numpy as np
def get_data():
    """
    Returns the data from the xlsx file
    """
    file_name = 'yeast.csv' 
    df = pd.read_csv(file_name, index_col=None)
    # print(df.head()) # print the first 5 rows
    return df

C = 10

D = 8
# p = 8

md = get_data()


# drop the 0 column
# ?????????
md = md.drop(columns=['DBName'])

md.dropna(inplace = True)
# print(md.shape)
md.replace('?', 0, inplace = True)

print(md.head(10))

# print(md[0 : 1])

#print(columns[1])


      1     2     3     4    5    6     7     8 OutName
0  0.58  0.61  0.47  0.13  0.5  0.0  0.48  0.22     MIT
1  0.43  0.67  0.48  0.27  0.5  0.0  0.53  0.22     MIT
2  0.64  0.62  0.49  0.15  0.5  0.0  0.53  0.22     MIT
3  0.58  0.44  0.57  0.13  0.5  0.0  0.54  0.22     NUC
4  0.42  0.44  0.48  0.54  0.5  0.0  0.48  0.22     MIT
5  0.51  0.40  0.56  0.17  0.5  0.5  0.49  0.22     CYT
6  0.50  0.54  0.48  0.65  0.5  0.0  0.53  0.22     MIT
7  0.48  0.45  0.59  0.20  0.5  0.0  0.58  0.34     NUC
8  0.55  0.50  0.66  0.36  0.5  0.0  0.49  0.22     MIT
9  0.40  0.39  0.60  0.15  0.5  0.0  0.58  0.30     CYT


Convert continuous data to categorical one

In [3]:
noOfBins = 20
# print(md['DBName'])
# print(md.columns)
# print(md['1'])
for i in range(D):
    md["cat" + str(i + 1)] = pd.cut(md[str(i+1)].values, bins=noOfBins, labels= range(0, noOfBins))
md.drop(columns=['1', '2', '3', '4', '5', '6', '7', '8'], inplace=True)
last_column = md.pop('OutName')
md.insert(8, 'OutName', last_column)
md.head(20)
# gk = md.groupby('Outname')
# print(md[1].value_counts())


,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,OutName
0,10,11,6,2,0,0,13,4,MIT
1,7,12,6,5,0,0,14,4,MIT
2,11,11,7,2,0,0,14,4,MIT
3,10,7,9,2,0,0,14,4,NUC
4,6,7,6,10,0,0,13,4,MIT
5,8,6,8,3,0,12,13,4,CYT
6,8,9,6,12,0,0,14,4,MIT
7,8,7,9,3,0,0,15,6,NUC
8,9,8,11,7,0,0,13,4,MIT
9,6,5,9,2,0,0,15,5,CYT


In [4]:
X = md.iloc[:, 0:8].values.reshape(-1, 8)
# X = normalize(X)
# outputs = ['CYT', 'NUC', 'MIT', 'ME3', 'ME2', 'ME1', 'EXC', 'VAC', 'POX', 'ERL']

outputs = ['CYT', 'ERL', 'EXC', 'ME1', 'ME2', 'ME3', 'MIT', 'NUC', 'POX', 'VAC']

# add 1 to the first column
# X = np.insert(X, 0, 1, axis=1)
# np.random.shuffle(X)

print('X:', X)
print(X.shape)

# X = normalize(X)
# print(X)


M = X.shape[0]
Y = md.iloc[:, 8:9].values.reshape(-1, 1)

X: [[10 11 6 ... 0 13 4]
 [7 12 6 ... 0 14 4]
 [11 11 7 ... 0 14 4]
 ...
 [12 10 3 ... 0 15 4]
 [7 6 9 ... 0 14 7]
 [12 9 8 ... 0 14 4]]
(1484, 8)


In [5]:
y = np.zeros(len(Y))

for i in range(len(Y)):
    y[i] = outputs.index(Y[i][0])


In [6]:
ncValues = np.zeros(C)
piValues = []

for i in range(M):
    ncValues[int(y[i])] += 1
for c in range(C):
    piValues.append(ncValues[c]/M)

# piValues = np.ones(C)/C

print(sum(piValues))

# md.dtypes
md[['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8']] = md[['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8']].astype(int)

data_mod = md.groupby(['OutName']).sum()



# data_mod = data_mod.T


# data_no_out = md.drop(columns=['OutName'])
# total_data_per_class = data_no_out.sum()

axisSum = data_mod.sum(axis=1)

theta = []
alpha = 0.001

for d in range(D):
    theta.append((data_mod['cat' + str(d + 1)] + alpha )/ (axisSum + alpha*D))

# theta = np.asarray(theta)
# theta = theta.T
# class_count = np.array(C)

print(y.sum(axis=0))

# print('sum ',data_mod.sum(axis=1))
# axes = md.axes
# columns = md.columns

# for i in range(D):
#     print(sum(theta[i]))

# theta = np.asanyarray(theta)
# theta = normalize(theta)
# theta_dic = theta.to_dict()
# print(theta_dic)

1.0
6123.0


In [12]:
# make prediction
md_dic = md.to_dict()
# print(md_dic)
# for d in range(D):
#     print(np.log(1 + md_dic['cat' + str(d + 1)][0]))

testSet = X
prediction = np.zeros(len(testSet))

# prediction[0] = 'asd'

for i in range(len(testSet)): 
    prbAllClass = np.zeros(C)
    for c in range(C):
        prbAllClass[c] = np.log(piValues[c])
        for d in range(D):
            first = md_dic['cat' + str(d + 1)][i]
            
            # first = np.log(1 + md_dic['cat' + str(d + 1)][i])
            second = np.log(theta[d][c]) # ['cat' + str(d + 1)][outputs[c]])
            
            prbAllClass[c] += first * second
            # print( i, prbAllClass[c], first, second)
    print(prbAllClass)
    prediction[i] = np.argmax(prbAllClass)
    print(prediction[i])


print("accuracy: ", (prediction == y[:len(testSet)]).sum()/len(testSet))

[-78.9827189  -95.24362887 -80.78258599 -81.6523572  -80.93746555
 -80.19750905 -80.86067303 -79.73778153 -90.30067272 -80.35574899]
0.0
[ -84.08109378 -101.37175675  -86.59369722  -87.33496648  -86.79125401
  -84.71319774  -84.38840302  -84.6137432   -95.67103846  -86.18257112]
0.0
[ -83.70890228 -101.12574871  -85.78425534  -87.14889596  -86.03429424
  -85.51792916  -85.86866154  -84.59203861  -95.662938    -85.17712708]
0.0
[-78.2002978  -97.58405531 -82.81678189 -85.64929119 -83.03419616
 -82.01831128 -81.00908917 -78.85345532 -90.28796173 -80.95562699]
0.0
[ -84.17037834 -102.35681762  -88.21506811  -89.03006369  -87.70111953
  -85.53275821  -82.11379512  -84.34398852  -95.4945669   -87.93726018]
6.0
[-161.8139275  -245.62571988 -250.87927784 -256.44498431 -254.71348075
 -263.99617345 -140.40102986 -274.04190688 -104.58849886 -245.05545367]
8.0
[ -97.38378418 -116.02146369 -100.08804301 -100.53911427  -99.63334378
  -98.26791355  -94.23005815  -97.83646864 -109.55002911 -100.83051

In [8]:
print(md_dic['cat1'][0])
print(md_dic['cat2'][5])
print(theta[0]['CYT'])
print(theta[0]['ERL'])
print(theta[0][1])

10
6
0.1745641723724656
0.1859284235492754
0.1859284235492754


In [9]:
# from sklearn.model_selection import train_test_split
# from sklearn.naive_bayes import ComplementNB

# X_train, X_test, Y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
# gnb = ComplementNB()
# gnb.fit(X_train, Y_train)
# y_pred = gnb.predict(X_test)
# print("Number of mislabeled points out of a total %d points : %d"
#       % (X_test.shape[0], (y_test != y_pred).sum()))

# from sklearn import metrics

# # Model Accuracy, how often is the classifier correct?
# print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [10]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

gnb = MultinomialNB()
gnb.fit(X_train, Y_train)

# X_test = X[:50]
y_pred = gnb.predict(X_test)
# y_test = y[:50]


print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))



from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Number of mislabeled points out of a total 594 points : 259
Accuracy: 0.563973063973064
